In [1]:
import socket
import ssl
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.serialization import load_pem_private_key, load_pem_public_key
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.serialization import load_pem_private_key
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives import serialization
import os


In [2]:

def generate_rsa_keypair():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    return private_key

def generate_dh_keypair():
    parameters = dh.generate_parameters(generator=2, key_size=2048)
    private_key = parameters.generate_private_key()
    public_key = private_key.public_key()
    return private_key, public_key

def derive_dh_shared_key(private_key, public_key):
    shared_key = private_key.exchange(public_key)
    return shared_key

def rsa_encrypt(public_key, plaintext):
    ciphertext = public_key.encrypt(
        plaintext.encode(),
        padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None)
        )
    return ciphertext

def rsa_decrypt(private_key, ciphertext):
    plaintext = private_key.decrypt(
    ciphertext,
    padding.OAEP( mgf=padding.MGF1(algorithm=hashes.SHA256()),
    algorithm=hashes.SHA256(),
    label=None))
    return plaintext.decode()

def aes_encrypt(key, plaintext):
    iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(plaintext.encode()) + encryptor.finalize()
    return iv + ciphertext

def aes_decrypt(key, ciphertext):
    iv = ciphertext[:16]
    ciphertext = ciphertext[16:]
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    return plaintext.decode()

def start_client():
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    context = ssl.SSLContext(ssl.PROTOCOL_TLS_CLIENT)
    context.load_verify_locations(cafile="ca.crt")
    client_socket.connect(("localhost", 8443))

    with context.wrap_socket(client_socket, server_hostname="localhost") as ssock:
        
        print("SSL/TLS connection established.")

        ssock.send("Please choose encryption method: 1 for RSA or 2 for AES (Diffie-Hellman)".encode())

        encryption_choice = ssock.recv(1024).decode()

        if encryption_choice == "1":
            # RSA encryption and PKI key exchange
            private_key = generate_rsa_keypair()
            public_key = private_key.public_key()
            ssock.send(public_key.public_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PublicFormat.SubjectPublicKeyInfo
            ))

            data = ssock.recv(2048)
            message = rsa_decrypt(private_key, data)
            print(f"Decrypted RSA message: {message}")

        elif encryption_choice == "2":
            # AES encryption and Diffie-Hellman key exchange
            bob_private_key, bob_public_key = generate_dh_keypair()
            ssock.send(bob_public_key.public_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PublicFormat.SubjectPublicKeyInfo
            ))

            received_public_key = ssock.recv(2048)
            received_public_key = serialization.load_pem_public_key(received_public_key, backend=default_backend())
            shared_key = derive_dh_shared_key(bob_private_key, received_public_key)

            data = ssock.recv(1024)
            message = aes_decrypt(shared_key, data)
            print(f"Decrypted AES message: {message}")



In [3]:
if __name__ == "__main__":
    start_client()
#openssl req -x509 -newkey rsa:4096 -keyout ca.key -out ca.crt -days 365
    """
    info for ca.crt
    PEM pass phrase: 03455
    country name: PK
    state or province name: Punjab
    locality name: Rawalpindi
    organization name: FAST
    organizational unit name: Computing
    common name: FARQ
    email address: i200621@nu.edu.pk
    """



ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host